In [3]:
import time

def mesure(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"'{func.__name__}': {execution_time:.6f} seconds.")
        return result
    return wrapper

In [4]:
import matplotlib.pyplot as plt
from svgpathtools import Line, QuadraticBezier, CubicBezier, Arc, Path
from svgpathtools import svg2paths, svg2paths2, wsvg
Path.iscontained = False

@mesure
def char2paths(char): # Implement character with multiple path, maybe ussing parsing

    plt.figure(figsize=(4, 4))
    plt.text(0.5, 0.5, char[0], fontsize=200, ha='center', va='center')
    plt.axis('off')
    filename='.matplot'
    plt.savefig(filename, format='svg')
    plt.close()
    
    paths, _, _ = svg2paths2(filename)
    paths = paths[1].continuous_subpaths()
    wsvg(paths, filename='svgtools.svg')
    
    return paths #list

@mesure
def set_contained_attr(paths: list) -> None:
    for path in paths:
        path.iscontained = any(path.is_contained_by(compare) for compare in paths if compare != path)



In [144]:
from svgpathtools import Line, QuadraticBezier, CubicBezier, Arc, Path
import numpy as np

@mesure
def offset_segment(seg,distance,res=5):
    
    if isinstance(seg,Line):
        offset_vector = distance * seg.normal(0)
        offset_seg = Line(seg.point(0) + offset_vector,seg.point(1) + offset_vector)
        return offset_seg

    elif isinstance(seg,Arc) or isinstance(seg,QuadraticBezier) or isinstance(seg,Line):

        offset_seg = Path()
        curr = seg.point(0) + distance * seg.normal(0)
        curr_tan = seg.unit_tangent(0)
        for k in range(1,res+1):
            t=k / res
            next = seg.point(t) + distance * seg.normal(t)
            next_tan = seg.unit_tangent(t)
            
            # Having curr/next as start/end  of each segment,
            # vectors curr_tan/next_tan pointing to a common point
            # to be used as refference for offset segment as bezier
            if curr_tan.real == 0:
                X = curr.real
                Y = (next_tan.imag/next_tan.real) * (X - next.real) + next.imag
            
            elif next_tan.real == 0:
                X = next.real
                Y = (curr_tan.imag/curr_tan.real) * (X - curr.real) + curr.imag
            
            else:
                Ax = curr.real; Ay = curr.imag
                Bx = next.real; By = next.imag

                ma = curr_tan.imag/curr_tan.real
                mb = next_tan.imag/next_tan.real

                # solving for lines intersection
                X = ((By-Ay) - (mb * Bx - ma * Ax))/(ma-mb)
                Y = (ma*mb * (Bx-Ax) - (By*ma-Ay*mb))/(mb-ma)
            
            control = complex(X,Y)
            new_element = QuadraticBezier(curr,control,next)
            # if offset_seg: print(new_element, offset_seg[-1])
            
            # print(offset_seg.intersect(new_element))
            # Before appending new_element, check if it intersects with some
            # non consecutive previous element, if so, delete all elements between
            # both of them, chop tails and check for continuity.
            offset_seg.append(new_element)
            curr = next
            curr_tan = next_tan
        
        return offset_seg            
             
    else:
        raise TypeError("Input `segment` should be a Line, "
                        "QuadraticBezier, CubicBezier, or Arc object.")

@mesure
def quad_intersect(qbez1: QuadraticBezier,qbez2: QuadraticBezier):

    x01, x11, x21 = qbez1.start.real, qbez1.control.real, qbez1.end.real
    x02, x12, x22 = qbez2.start.real, qbez2.control.real, qbez2.end.real
    y01, y11, y21 = qbez1.start.imag, qbez1.control.imag, qbez1.end.imag
    y02, y12, y22 = qbez2.start.imag, qbez2.control.imag, qbez2.end.imag


    A1, A2 = x01 - 2*x11 + x21, x02 - 2*x12 + x22
    C1, C2 = y01 - 2*y11 + y21, y02 - 2*y12 + y22
    B1, B2 = 2*(x11 - x01), 2*(x12 - x02) 
    D1, D2 = 2*(y11 - y01), 2*(y12 - y02)
    
    A, C, F, H = abs(A1) ** 0.5, abs(A2) ** 0.5, abs(C1) ** 0.5, abs(C2) ** 0.5
    B, D, G, I = B1/(2*A), B2/(2*C), D1/(2*F), D2/(2*H)

    E = (B1**2)/(4*A1) - (B2**2)/(4*A2) + x02 - x01
    J = (D1**2)/(4*C1) - (D2**2)/(4*C2) + y02 - y01

    P = (abs(H**2 * A**2 - C**2 * F**2))**0.5
    Q = (A*B - F*G)/P
    R = -((A*B - F*G)**2) * (H**2 * A**2 - C**2 * F**2) + H**2 * B**2 - H**2 * E - C**2 * G**2 + C**2 * J - (H*D - I*C)**2

    S = 2*C*(H*D - I*C)

    # P = (abs(F**2 * C**2 + A**2 * H**2))**0.5
    # Q = (F*C*D + A*H*I)/P
    # R = Q**2 + D**2 + I**2 + E*F**2 - J*A**2 - (B*F - A*G)**2
    # S = 2*A*(B*F - A*G)
    
    a = P**4
    b = 4 * Q * P**3
    c = 6 * P**2 * Q**2 + 2 * R * P**2 + S**2 * F**2
    d = 4 * P * Q**3 + 4*R*P*Q + 2 * S**2 * F * G
    e = (Q**2 + R)**2 + (S**2)*(J-G**2)

    # a = P**4
    # b = 4 * Q * P**3
    # c = 6 * P**2 * Q**2 + 2 * R * P**2 + S**2 * H**2
    # d = 4 * P * Q**3 + 4 * R * P * Q + 2 * S**2 * H * I
    # e = 2 * R * Q**2 + R**2 + S**2 * (I**2 - J)

    roots = np.roots([a,b,c,d,e])

    print(*roots, sep='\n\n')

In [145]:
from svgpathtools import Line, QuadraticBezier, CubicBezier, Arc, Path
from svgpathtools import svg2paths, svg2paths2, wsvg, disvg
seg1 = Line(200+300j, 250+350j)
seg2 = Arc(start=200+300j,radius=90+40j,rotation=0,large_arc=True,sweep=True,end=250+350j)
seg3 = QuadraticBezier(210+200j,300+400j,300+300j)
off = offset_segment(seg3,10)

intersected_segs = Path(off[2],off[4])
quad_intersect(*intersected_segs)
# print(offset_intersects(intersected_segs))
# offset2 = offset_segment(seg2,10,res=1000)
# print(offset2)
# aprox2 = next(offset2.as_cubic_curves(1))
# print(path)
# offset_path = Path(offset_segments)
path = Path(*intersected_segs)
disvg(path,filename="intersections.svg",openinbrowser=False)

'offset_segment': 0.000000 seconds.
(2.7360689280984234+3.374566764064522j)

(2.7360689280984234-3.374566764064522j)

(-2.934085135180122+0j)

(-1.671264969470404+0j)
'quad_intersect': 0.000000 seconds.


In [5]:
@mesure
def run():
    paths = char2paths('%')
    set_contained_attr(paths)

run()

'char2paths': 0.114476 seconds.
'set_contained_attr': 0.034160 seconds.
'run': 0.149636 seconds.


In [6]:
def arc_offset(arc: Arc,distance: float):
    start = arc.start + distance * arc.normal(0)
    end = arc.end + distance * arc.normal(1)
    radius = arc.radius
    radius = 1.2*arc.radius
    rotation = arc.rotation
    large_arc = arc.large_arc
    sweep = arc.sweep
    offset_arc = Arc(start,radius,rotation,large_arc,sweep,end)
    offset_arc.center = arc.center
    return offset_arc

In [7]:
import numpy as np
seg3 = QuadraticBezier(200+200j,300+400j,300+300j)
seg3_tan = seg3.unit_tangent(1)
np.tan(np.angle(seg3_tan))

-1.633123935319537e+16

In [8]:
from svgpathtools import Path, CubicBezier, Arc
from svgpathtools import svg2paths, svg2paths2, wsvg, disvg
import numpy as np

def ellipse_parametric(a, b, t):
    x = a * np.cos(t)
    y = b * np.sin(t)
    return x, y

# Semiejes de la elipse
a = 100  # Semieje mayor
b = 50   # Semieje menor

# Número de puntos para aproximar la elipse
num_points = 4

# Crear el vector de valores de t
t_values = np.linspace(0, 2*np.pi, num_points, endpoint=False)

# Crear la lista de puntos para la curva paramétrica de la elipse
points = [ellipse_parametric(a, b, t) for t in t_values]

# Crear la ruta paramétrica utilizando la lista de puntos
path = [CubicBezier(start=points[i], control1=points[i], control2=points[i+1], end=points[i+1]) for i in range(num_points-1)]

path = Path(*path)

print(*path,sep='\n')
disvg(path,filename="intersections.svg",openinbrowser=False)

CubicBezier(start=(100.0, 0.0), control1=(100.0, 0.0), control2=(6.123233995736766e-15, 50.0), end=(6.123233995736766e-15, 50.0))
CubicBezier(start=(6.123233995736766e-15, 50.0), control1=(6.123233995736766e-15, 50.0), control2=(-100.0, 6.123233995736766e-15), end=(-100.0, 6.123233995736766e-15))
CubicBezier(start=(-100.0, 6.123233995736766e-15), control1=(-100.0, 6.123233995736766e-15), control2=(-1.8369701987210297e-14, -50.0), end=(-1.8369701987210297e-14, -50.0))


AttributeError: 'tuple' object has no attribute 'real'

In [86]:
for t in range(20000000):
    a = 10000**0.5

a

100.0

In [87]:
for t in range(20000000):
    a = np.sqrt(10000)

a

100.0